In [19]:
import requests
import pandas as pd 
import json
import calendar
import hvplot.pandas
from pathlib import Path
import seaborn as sns
%matplotlib inline
from bokeh.models.formatters import DatetimeTickFormatter

pd.set_option("display.max_rows", 10, "display.max_columns", None)

In [20]:
path_2019 = Path("../FInCen CSV Data/FinCen SAR Stats 2019 (Months).csv")
sar_stats_2019 = pd.read_csv(path_2019, thousands=",", index_col = 'Year Month')
sar_stats_2019

,State,Industry,Suspicious Activity,Count
Year Month,,,,
2019 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Check,1
2019 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,1
2019 January,Alabama,Casino/Card Club - Tribal Authorized Casino,[Total],2
2019 January,Alabama,Depository Institution,ACH,126
2019 January,Alabama,Depository Institution,Advance Fee,4
...,...,...,...,...
2019 December,Wyoming,Securities/Futures,[Total],1
2019 December,Wyoming,[Total],[Total],60
2019 December,[Total],[Total],[Total],103088


In [21]:
sar_stats_2019_clean = sar_stats_2019.loc[sar_stats_2019["Suspicious Activity"] != "[Total]"]
sar_stats_2019_clean = sar_stats_2019_clean.loc[sar_stats_2019_clean["Industry"] != "[Total]"]
sar_stats_2019_clean = sar_stats_2019_clean.loc[sar_stats_2019_clean["State"] !="[Total]"]
sar_stats_2019_clean

,State,Industry,Suspicious Activity,Count
Year Month,,,,
2019 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Check,1
2019 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,1
2019 January,Alabama,Depository Institution,ACH,126
2019 January,Alabama,Depository Institution,Advance Fee,4
2019 January,Alabama,Depository Institution,Business Loan,1
...,...,...,...,...
2019 December,Wyoming,Depository Institution,Wire,6
2019 December,Wyoming,Money Services Business (MSB),Advance Fee,2
2019 December,Wyoming,Money Services Business (MSB),Other Fraud (Type),13


In [22]:
sar_stats_2019_clean = sar_stats_2019_clean.drop(columns= 'Suspicious Activity')
sar_stats_2019_clean

,State,Industry,Count
Year Month,,,
2019 January,Alabama,Casino/Card Club - Tribal Authorized Casino,1
2019 January,Alabama,Casino/Card Club - Tribal Authorized Casino,1
2019 January,Alabama,Depository Institution,126
2019 January,Alabama,Depository Institution,4
2019 January,Alabama,Depository Institution,1
...,...,...,...
2019 December,Wyoming,Depository Institution,6
2019 December,Wyoming,Money Services Business (MSB),2
2019 December,Wyoming,Money Services Business (MSB),13


In [23]:
sar_stats_2019_clean = sar_stats_2019_clean.loc[sar_stats_2019_clean["Industry"] == 'Depository Institution']
sar_stats_2019_clean

,State,Industry,Count
Year Month,,,
2019 January,Alabama,Depository Institution,126
2019 January,Alabama,Depository Institution,4
2019 January,Alabama,Depository Institution,1
2019 January,Alabama,Depository Institution,498
2019 January,Alabama,Depository Institution,49
...,...,...,...
2019 December,Wyoming,Depository Institution,1
2019 December,Wyoming,Depository Institution,1
2019 December,Wyoming,Depository Institution,1


In [24]:
sar_stats_2019_clean = sar_stats_2019_clean.reset_index()
sar_stats_2019_clean['Year Month'] = pd.to_datetime(sar_stats_2019_clean['Year Month'], infer_datetime_format = True)
sar_stats_2019_clean['Count'] = sar_stats_2019_clean['Count'].astype(float)

sar_stats_2019_clean.dtypes

Year Month    datetime64[ns]
State                 object
Industry              object
Count                float64
dtype: object

In [25]:
grouped_stats_state = sar_stats_2019_clean[["State", "Count", "Year Month"]]

grouped_stats_state = grouped_stats_state.groupby(['Year Month', 'State']).sum()
grouped_stats_state

Count
Year Month State                 
2019-01-01 Alabama          961.0
           Alaska            64.0
           Arizona          533.0
           Arkansas         203.0
           California      6799.0
...                           ...
2019-12-01 Virginia       12271.0
           Washington       701.0
           West Virginia     89.0
           Wisconsin        429.0
           Wyoming           34.0

[680 rows x 1 columns]

In [189]:
grouped_stats_state = grouped_stats_state.sort_index(level = 1, ascending = False)
grouped_stats_state

,,Reports
Year Month,State,
2019-12-01,Wyoming,34.0
2019-11-01,Wyoming,28.0
2019-10-01,Wyoming,43.0
2019-09-01,Wyoming,25.0
2019-08-01,Wyoming,26.0
...,...,...
2019-05-01,Alabama,949.0
2019-04-01,Alabama,901.0
2019-03-01,Alabama,969.0


In [26]:
grouped_stats_ym = sar_stats_2019_clean.groupby(['Year Month']).sum()
grouped_stats_ym

,Count
Year Month,
2019-01-01,70132.0
2019-02-01,70183.0
2019-03-01,79280.0
2019-04-01,68218.0
2019-05-01,71385.0
...,...
2019-08-01,73652.0
2019-09-01,72308.0
2019-10-01,81523.0


In [27]:
grouped_stats_ym = grouped_stats_ym.rename(columns = {'Count' : 'Total'})
grouped_stats_ym

,Total
Year Month,
2019-01-01,70132.0
2019-02-01,70183.0
2019-03-01,79280.0
2019-04-01,68218.0
2019-05-01,71385.0
...,...
2019-08-01,73652.0
2019-09-01,72308.0
2019-10-01,81523.0


In [28]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
grouped_stats_state = grouped_stats_state.rename(columns = {'Count' : 'Reports'})
grouped_stats_state

Reports
Year Month State                 
2019-01-01 Alabama          961.0
           Alaska            64.0
           Arizona          533.0
           Arkansas         203.0
           California      6799.0
...                           ...
2019-12-01 Virginia       12271.0
           Washington       701.0
           West Virginia     89.0
           Wisconsin        429.0
           Wyoming           34.0

[680 rows x 1 columns]

In [29]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
fraud_data = pd.merge(grouped_stats_state, grouped_stats_ym, left_on = 'Year Month',right_index=True)
fraud_data

Reports    Total
Year Month State                          
2019-01-01 Alabama          961.0  70132.0
           Alaska            64.0  70132.0
           Arizona          533.0  70132.0
           Arkansas         203.0  70132.0
           California      6799.0  70132.0
...                           ...      ...
2019-12-01 Virginia       12271.0  73501.0
           Washington       701.0  73501.0
           West Virginia     89.0  73501.0
           Wisconsin        429.0  73501.0
           Wyoming           34.0  73501.0

[680 rows x 2 columns]

In [30]:
fraud_data['Reporting Rate'] = round((fraud_data['Reports']/fraud_data['Total'])*100,3)
fraud_data

Reports    Total  Reporting Rate
Year Month State                                          
2019-01-01 Alabama          961.0  70132.0           1.370
           Alaska            64.0  70132.0           0.091
           Arizona          533.0  70132.0           0.760
           Arkansas         203.0  70132.0           0.289
           California      6799.0  70132.0           9.695
...                           ...      ...             ...
2019-12-01 Virginia       12271.0  73501.0          16.695
           Washington       701.0  73501.0           0.954
           West Virginia     89.0  73501.0           0.121
           Wisconsin        429.0  73501.0           0.584
           Wyoming           34.0  73501.0           0.046

[680 rows x 3 columns]

In [31]:
fraud_data_rate = fraud_data[['Reporting Rate']].copy()
formatter = DatetimeTickFormatter(months='%b %Y')
fraud_data_rate.hvplot(groupby='State' ,yformatter = '%.3f', xformatter=formatter)

:DynamicMap   [State]
   :Curve   [Year Month]   (Reporting Rate)

In [34]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
fraud_data_rate

Reporting Rate
Year Month State                        
2019-01-01 Alabama                 1.370
           Alaska                  0.091
           Arizona                 0.760
           Arkansas                0.289
           California              9.695
...                                  ...
2019-12-01 Virginia               16.695
           Washington              0.954
           West Virginia           0.121
           Wisconsin               0.584
           Wyoming                 0.046

[680 rows x 1 columns]